In [1]:
import serial
import time
import numpy as np

In [17]:
def send_command(cmd):
    l = cmd.strip()
    print l
    s.write(cmd + "\n")
    grbl_out = s.readline() # Wait for grbl response with carriage return
    print '> ' + grbl_out.strip()

def in_range(x, y):
    return ((0 <= x <= 240) and (0 <= y <= 170))
    
def move(x, y):
    if in_range(x, y):
        send_command("G01 X{} Y{}".format(-x, y))
    else:
        print("{}, {} not in range".format(x, y))
    
def move_quick(x, y):
    if in_range(x, y):
        send_command("G00 X{} Y{}".format(-x, y))
    else:
        print("{}, {} not in range".format(x, y))
    
def set_speed(speed):
    # in mm/sec
    send_command("F{}".format(speed))
    
def pen_down():
    send_command("S1000 M3")
    
def pen_up():
    send_command("S0 M5")
    
def set_unit():
    send_command("G21")    # set units in millimeters
    
def draw_line(x1, y1, x2, y2):
    move(x1, y1)
    pen_down()
    move(x2, y2)
    pen_up()
    
def arc(x, y, i, j, cw = True):
    if cw:
        c = "2"
    else:
        c = "3"
    send_command("G0{} X{} Y{} I{} J{}".format(c, -x, y, -i, j))
    
def draw_circle(x, y, r):
    if in_range(x-r, y-r) and in_range(x+r, y+r):
        move(x-r/2., y-r/2.)
        pen_down()
        arc(x-r/2., y-r/2., r/2., r/2.)
        pen_up()
    else:
        print("circle {},{},{} not in range".format(x, y, r))
        
def initialize(s):
    # Wake up grbl
    s.write("\r\n\r\n")
    time.sleep(5)   # Wait for grbl to initialize
    s.flushInput()  # Flush startup text in serial input

In [18]:
# Open grbl serial port
s = serial.Serial('/dev/tty.wchusbserial1420',115200)

initialize(s)

In [19]:
set_speed(1000)

F1000
> ok


In [15]:
move(0, 0)

G01 X0 Y0
> ok


In [13]:
pen_down()
pen_up()

S1000 M3
> ok
S0 M5
> ok


In [14]:
move(60, 60)

G01 X-60 Y60
> ok


In [75]:
pen_down()
send_command("G02 X-58 Y62 I2 J0")
pen_up()

S1000 M3
> ok
G02 X-58 Y62 I2 J0
> ok
S0 M5
> ok


In [77]:
# Close serial port
s.close()

In [34]:
for x_loc in range(0, 50, 10):
    for y_loc in range(0, 50, 10):
        shape = create_shape()
        for line in shape:
            x1,y1 = line[0]
            x2,y2 = line[1]
            draw_line(x_loc + x1*4, y_loc + y1*4, x_loc + x2*4, y_loc + y2*4)

G01 X-8 Y12
> ok
S1000 M3
> ok
G01 X-12 Y0
> ok
S0 M5
> ok
G01 X-4 Y12
> ok
S1000 M3
> ok
G01 X-8 Y12
> ok
S0 M5
> ok
G01 X-8 Y8
> ok
S1000 M3
> ok
G01 X-8 Y0
> ok
S0 M5
> ok
G01 X0 Y22
> ok
S1000 M3
> ok
G01 X-12 Y14
> ok
S0 M5
> ok
G01 X-12 Y18
> ok
S1000 M3
> ok
G01 X-4 Y22
> ok
S0 M5
> ok
G01 X-4 Y24
> ok
S1000 M3
> ok
G01 X-12 Y24
> ok
S0 M5
> ok
G01 X-8 Y20
> ok
S1000 M3
> ok
G01 X0 Y20
> ok
S0 M5
> ok
G01 X-12 Y28
> ok
S1000 M3
> ok
G01 X-12 Y32
> ok
S0 M5
> ok
G01 X-12 Y28
> ok
S1000 M3
> ok
G01 X-4 Y24
> ok
S0 M5
> ok
G01 X0 Y38
> ok
S1000 M3
> ok
G01 X-12 Y42
> ok
S0 M5
> ok
G01 X-12 Y42
> ok
S1000 M3
> ok
G01 X-4 Y38
> ok
S0 M5
> ok
G01 X-12 Y30
> ok
S1000 M3
> ok
G01 X-12 Y38
> ok
S0 M5
> ok
G01 X-4 Y52
> ok
S1000 M3
> ok
G01 X-8 Y52
> ok
S0 M5
> ok
G01 X-12 Y40
> ok
S1000 M3
> ok
G01 X-4 Y52
> ok
S0 M5
> ok
G01 X0 Y44
> ok
S1000 M3
> ok
G01 X-12 Y52
> ok
S0 M5
> ok
G01 X-14 Y4
> ok
S1000 M3
> ok
G01 X-14 Y12
> ok
S0 M5
> ok
G01 X-10 Y0
> ok
S1000 M3
> ok
G01 X-14 Y4
> ok
S

In [33]:
def create_shape():
    # select random points and connect them
    
    points = np.array([(i,j) for i in range(4) for j in range(4)], dtype='int,int')
    
    lines = []

    for i in range(np.random.randint(1,6)):
        line = np.random.choice(points, size = 2, replace=False)
        line = tuple(line)
        if not line in lines:
            lines.append(tuple(line))

    return list(lines)